# Ассоциативные правила
Метод поиска взаимосвязей в данных

![rules](https://www.saedsayad.com/images/AR_1.png)

**Rule (правило)** - Если купил Х, то купит и Y.

**Support** - Как часто набор элементов появляется в датасете. Рассматривают только топ сетов с максимальным значением параметра.

**Confidence** - Как часто Y покупалось с Х на самом деле.

**Lift** - Какова сила взаимосвязи элементов. Чем значение больше, тем лучше. При значении 1 - элемнты независимы.

In [ ]:
import pandas as pd

Данные [отсюда](https://www.kaggle.com/apmonisha08/market-basket-analysis).

In [ ]:
!head '../data/Groceries.csv'

In [ ]:
data = pd.read_csv('../data/Groceries.csv', header=None)

In [ ]:
data.shape

In [ ]:
transactions = []
for i in range(0, data.shape[0]):
    transactions.append([data.values[i,j] for j in range(0, data.shape[1]) if str(data.values[i,j])!='nan'])

In [ ]:
transactions[:10]

In [ ]:
# !pip install apyori
from apyori import apriori, dump_as_json

In [ ]:
result = list(apriori(transactions, min_support = 0.005, min_confidence = 0.2, min_lift = 4, max_length = 3))

In [ ]:
result

In [ ]:
import shutil, os 
from io import StringIO
import json 
output = []
for RelationRecord in result:
    o = StringIO()
    dump_as_json(RelationRecord, o)
    output.append(json.loads(o.getvalue()))
df = pd.DataFrame(output)
df.sort_values(by='support', inplace=True, ascending=False)
# и взгялнем на итоги
pd.set_option('display.max_colwidth', -1)
df

In [ ]:
df[['0', '1']] = df['ordered_statistics'].apply(pd.Series)

In [ ]:
df

In [ ]:
df0 = df['0'].apply(pd.Series)
df1 = df['1'].apply(pd.Series)
df.drop(['0', '1'], axis=1, inplace=True)

In [ ]:
df0

In [ ]:
df1

In [ ]:
df1.dropna(how='all', axis=1, inplace=True)
df1

In [ ]:
df0 = df.merge(df0, left_index=True, right_index=True)

In [ ]:
df1 = df.merge(df1, left_index=True, right_index=True)

In [ ]:
df1

In [ ]:
df1.dropna(how='any', axis=0, inplace=True)

In [ ]:
df1

In [ ]:
df = df0.reset_index(drop=True).append(
                                df1.reset_index(drop=True)
                                ).reset_index(drop=True).drop(
                                                        'ordered_statistics', 
                                                        axis=1).sort_values(by='support',
                                                                           ascending=False).reset_index(
                                                                                            drop=True)

In [ ]:
df